In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import json
from aml_storage import Labels, Block, Descriptor
from utils.builder import DescriptorBuilder
import ase.io
from itertools import product
from utils.clebsh_gordan import ClebschGordanReal
from utils.hamiltonians import fix_pyscf_l1, dense_to_blocks, blocks_to_dense, couple_blocks, decouple_blocks
import matplotlib.pyplot as plt
from utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
from utils.hamiltonians import *
import copy

In [3]:
frames = ase.io.read("data/water-hamiltonian/water_coords_1000.xyz",":1000")
for f in frames:
    f.cell = [100,100,100]
    f.positions += 50

In [4]:
jorbs = json.load(open('data/water-hamiltonian/orbs_def2_water.json', "r"))
orbs = {}
zdic = {"O" : 8, "H":1}
for k in jorbs:
    orbs[zdic[k]] = jorbs[k]

In [5]:
hams = np.load("data/water-hamiltonian/water_fock.npy", allow_pickle=True)
for i, f in enumerate(frames):
    hams[i] = fix_pyscf_l1(hams[i], f, orbs)

In [6]:
cg = ClebschGordanReal(4)

In [7]:
rascal_hypers = {
    "interaction_cutoff": 3.5,
    "cutoff_smooth_width": 0.5,
    "max_radial": 3,
    "max_angular": 2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
}

In [8]:
spex = RascalSphericalExpansion(rascal_hypers)
rhoi = spex.compute(frames)

In [9]:
def get_lm_slice(hypers):
    lm_slices = []
    start = 0
    for l in range(hypers["max_angular"] + 1):
        stop = start + 2 * l + 1
        lm_slices.append(slice(start, stop))
        start = stop
    return lm_slices

In [10]:
from rascal.representations import SphericalExpansion

In [11]:
pairs = RascalPairExpansion(rascal_hypers)
gij = pairs.compute(frames)

In [12]:
def compute_rho0ij(hypers, frames):
    lmax= hypers["max_angular"]
    pairs = RascalPairExpansion(hypers)
    gij = pairs.compute(frames)
    new_sparse = Labels(
    names=["sigma", "L", "nu"],
    values=np.array(
        [
            [1, l, 0]
            for l in range(hypers["max_angular"] + 1)                    
        ],
        dtype=np.int32,
            ),
        )
    blocks=[]
    for idx, block in gij:
        blocks.append(block.copy())
    return Descriptor(new_sparse, blocks)

In [13]:
hypers=rascal_hypers

In [56]:
old_rho0ij=compute_rho0ij(hypers, frames)

In [57]:
# works for just water

In [58]:
def compute_rho0ij_builder(hypers, frames):
    if hypers["compute_gradients"]:
        raise Exception("Pair expansion with gradient is not implemented")

    max_atoms = max([len(f) for f in frames])
    actual_global_species = list(
    map(int, np.unique(np.concatenate([f.numbers for f in frames]))))
    calculator = SphericalExpansion(**hypers)
    manager = calculator.transform(frames)
    info = manager.get_representation_info()
    global_species = list(range(max_atoms))
    
    hypers_ij= copy.deepcopy(hypers)
    hypers_ij["global_species"] = global_species
    hypers_ij["expansion_by_species_method"] = "user defined"
    lm_slices = get_lm_slice(hypers)

    ijframes = []
    for f in frames:
        ijf = f.copy()
        ijf.numbers = global_species[:len(f)]
        ijframes.append(ijf)

    calculator = SphericalExpansion(**hypers_ij)
    gij_expansion=calculator.transform(ijframes).get_features(calculator).reshape(len(ijframes), len(ijf), max_atoms, hypers_ij["max_radial"], -1) #TODO: change for differet len
#     print(gij_expansion.shape)
    
    feat_builder= DescriptorBuilder(["block_type", "L", "nu", "sigma",  "center_species", "neighbor_species"], ["structure", "center_i", "center_j"], ["mu"], ["n"])

    pair_loc=[]
    lmax = hypers["max_angular"]
    for sp_i in actual_global_species:
        for sp_j in actual_global_species:
            center_species_mask = np.where(info[:, 2] == sp_i)[0]
            neighbor_species_mask = np.where(info[:, 2] == sp_j)[0]
            for i, (struct_i, atom_i) in enumerate(info[center_species_mask[:]][...,:-1]):
                for j, (struct_j, atom_j) in enumerate(info[neighbor_species_mask[:]][...,:-1]):
                    if not (struct_i==struct_j):
                        continue
                    if atom_i==atom_j:
                            block_type = 0  # diagonal
                    elif sp_i==sp_j:
                        block_type = 1  # same-species
                    else:
                        block_type = 2  # different species
                    if [struct_i, atom_j, atom_i] not in pair_loc:
                        pair_loc.append([struct_i, atom_i, atom_j])

                    for l in range(lmax+1):
                        block_idx=(block_type, l, 0, 1, sp_i, sp_j)
                        if block_idx not in feat_builder.blocks:
                            block = feat_builder.add_block(sparse=block_idx, features=np.asarray([list(range(hypers["max_radial"]))], dtype=np.int32).T, 
                                components=np.asarray([list(range(-l, l+1))], dtype=np.int32 ).T )  
                            
                        
                            if block_type == 1:
                                block_asym = feat_builder.add_block(sparse=(-1,)+block_idx[1:], features=np.asarray([list(range(hypers["max_radial"]))], dtype=np.int32).T,
                                components=np.asarray([list(range(-l, l+1))], dtype=np.int32 ).T )                                                    
                        else:                        
                            block = feat_builder.blocks[block_idx]
                            if block_type == 1:
                                block_asym = feat_builder.blocks[(-1,)+block_idx[1:]]

                        block_data =gij_expansion[struct_i][atom_i%3, atom_j%3, :, lm_slices[l]].T #TODO: change for not water
                        #POSSIBLE replacement:(atom_i-sum(info[:struct_i,:].axis=1))%len(info[np.where(info[:,:,0]==struct_i)])
                        if block_type == 1:
                            if atom_i%3 <=atom_j%3:
                                block_data_ji = gij_expansion[struct_i][atom_j%3, atom_i%3, :, lm_slices[l]].T                  
                                block.add_samples(labels=np.asarray([[struct_i, atom_i%3, atom_j%3]], dtype=np.int32), data=(block_data+block_data_ji).reshape((1,-1,block_data.shape[1]))/np.sqrt(2) )
                                block_asym.add_samples(labels=np.asarray([[struct_i, atom_i%3, atom_j%3]], dtype=np.int32), data=(block_data-block_data_ji).reshape((1,-1,block_data.shape[1]))/np.sqrt(2) )
                        else:
#                       
                            block.add_samples(labels=np.asarray([[struct_i, atom_i%3, atom_j%3]], dtype=np.int32), data=block_data.reshape(1, -1, block_data.shape[1]))
    #                     

    return feat_builder.build()

In [59]:
rho0ij=compute_rho0ij_builder(hypers, frames)

In [60]:
# for even L's block_type -1 should be zero 
# for odd L's block_type 1 should be zero 
# block_type=0 - all values should be the same
#block_type=2, interchanging center and neighbor species should give equivalent results
rho0ij.block(block_type=2, L=0, nu=0, sigma=1, center_species=8, neighbor_species=1).values

array([[[ 5.81896807e-03, -2.63814570e-04, -3.25631700e-04]],

       [[ 4.82603864e-03,  5.83956974e-04, -3.51296448e-04]],

       [[ 5.80991748e-03, -2.55701240e-04, -3.26060736e-04]],

       ...,

       [[ 4.94725398e-03,  4.85315783e-04, -3.50608169e-04]],

       [[ 4.01298963e-03,  1.20557367e-03, -3.35942582e-04]],

       [[ 5.47491229e-03,  3.98296362e-05, -3.39516669e-04]]])

In [62]:
# to verify with the rascal spherical expansion- find indices of the relevant samples
idx=[]
i=0
for (struct, center_i, center_j, species_i, species_j) in old_rho0ij.block(L=0).samples:
    
    if center_i==center_j: 
        idx.append(i)
    i+=1

In [63]:
old_rho0ij.block(L=0).values[idx]

array([[[ 0.01039805, -0.00505374,  0.0002431 ]],

       [[ 0.01039805, -0.00505374,  0.0002431 ]],

       [[ 0.01039805, -0.00505374,  0.0002431 ]],

       ...,

       [[ 0.01039805, -0.00505374,  0.0002431 ]],

       [[ 0.01039805, -0.00505374,  0.0002431 ]],

       [[ 0.01039805, -0.00505374,  0.0002431 ]]])

# rho1ij

In [ ]:
rho1ij=

# CRAP

In [58]:
def hamiltonian_rep(frames, hypers, nu):
    if nu>0:
        raise ValueError("nu>0 not implemented")
    nu0_desc = compute_rho0ij(hypers, frames)
    
    sparse = (block_type, center_sp, neighbor_sp, L, sigma)

In [313]:
gij_expansion=calculator.transform(ijframes).get_features(calculator).reshape(len(ijframes), len(ijf), max_atoms, hypers["max_radial"], -1)


In [389]:
feat_builder= DescriptorBuilder(["block_type", "L", "nu", "sigma",  "center_species", "neighbor_species"], ["structure", "center_i", "center_j"], ["mu"], ["n"])

pair_loc=[]
lmax = hypers["max_angular"]
for sp_i in actual_global_species:
    for sp_j in actual_global_species:
        center_species_mask = np.where(info[:, 2] == sp_i)[0]
        neighbor_species_mask = np.where(info[:, 2] == sp_j)[0]
        for i, (struct_i, atom_i) in enumerate(info[center_species_mask[:]][...,:-1]):
            for j, (struct_j, atom_j) in enumerate(info[neighbor_species_mask[:]][...,:-1]):
                if not (struct_i==struct_j):
                    continue
                if atom_i==atom_j:
                        block_type = 0  # diagonal
                elif sp_i==sp_j:
                    block_type = 1  # same-species
                else:
                    block_type = 2  # different species
                if [struct_i, atom_j, atom_i] not in pair_loc:
                    pair_loc.append([struct_i, atom_i, atom_j])

                for l in range(lmax+1):
                    block_idx=(block_type, l, 0, 1, sp_i, sp_j)
                    if block_idx not in feat_builder.blocks:
                        block = feat_builder.add_block(sparse=block_idx, features=np.asarray([list(range(hypers["max_radial"]))], dtype=np.int32).T, 
                            components=np.asarray([list(range(-l, l+1))], dtype=np.int32 ).T )                                                    
#                         if block_type == 1:
#                             block_asym = feat_builder.add_block(sparse=(-1,)+block_idx[1:], features=np.asarray([list(range(hypers["max_radial"]))], dtype=np.int32).T,
#                             components=np.asarray([list(range(-l, l+1))], dtype=np.int32 ).T )                                                    
                    else:                        
                        block = feat_builder.blocks[block_idx]
#                         if block_type == 1:
#                             block_asym = feat_builder.blocks[(-1,)+block_idx[1:]]

                    block_data =gij_expansion[struct_i][atom_i%3, atom_j%3, :, lm_slices[l]].T
#                     if block_type == 1:
#                         kj_offset = kj-kj_base
#                         ki_offset = ki-ki_base
#                         block_data_ji = np.asarray(ham[kj_base+ki_offset:kj_base+ki_offset+2*li+1, ki_base+kj_offset:ki_base+kj_offset+2*lj+1])                        
#                         block.add_samples(labels=[(A,i,j, ki_base, kj_base)], data=(block_data_ij+block_data_ji).reshape((1,-1,1))/np.sqrt(2) )
#                         block_asym.add_samples(labels=[(A,i,j,ki_base, kj_base)], data=(block_data_ij-block_data_ji).reshape((1,-1,1))/np.sqrt(2) )
#                     else:
                    block.add_samples(labels=np.asarray([[struct_i, atom_i%3, atom_j%3]], dtype=np.int32), data=block_data.reshape(1, -1, block_data.shape[1]))
#                     block.add_features(labels=[(n,)], data=block_data_ij.reshape((1,-1,1))

In [390]:
aa=feat_builder.build()

In [398]:
aa.block(block_type=0, L=1, nu=0, sigma=1, center_species=1, neighbor_species=1).values.shape

(2000, 3, 3)

In [356]:
gij_expansion[struct_i][atom_i, atom_j, :, lm_slices[0]].reshape(1, -1, block_data.shape[1]).shape

(1, 1, 3)

In [350]:
np.asarray([list(range(hypers["max_radial"]))], dtype=np.int32).T#.shape

array([[0],
       [1],
       [2]], dtype=int32)

In [305]:
block.add_samples(labels=np.asarray([[struct_i, atom_i, atom_j]], dtype=np.int32), values=block_data)

In [249]:
blocks = []
for idx, (l, nu, sigma, sp_i, sp_j, ) in enumerate(sparse):
    center_species_mask = np.where(info[:, 2] == sp_i)[0]
    neighbor_species_mask = np.where(info[:, 2] == sp_j)[0]

    pair_loc=[]
    for i, (struct_i, atom_i) in enumerate(info[center_species_mask[:]][...,:-1]):
        for j, (struct_j, atom_j) in enumerate(info[neighbor_species_mask[:]][...,:-1]):
            if struct_i==struct_j:
                if [struct_i, atom_j, atom_i] not in pair_loc:
                    pair_loc.append([struct_i, atom_i, atom_j])
                    
    samples = Labels(
        names=["structure", "center_i", "center_j"],
        values=np.copy(pair_loc).astype(np.int32),
    )
    
    
    
    components = Labels(
                names=["mu"],
                values=np.array([[m] for m in range(-l, l + 1)], dtype=np.int32),
            )
    
    block_data = gij.block(spherical_harmonic_l=l).samples[]
    block = Block(
                values=np.copy(block_data),
                samples=samples,
                components=components,
#                 features=features,
            )
    blocks.append(block)
Descriptor(sparse, blocks)

TypeError: __init__() missing 2 required positional arguments: 'values' and 'features'

In [361]:
sp_j=1
sp_i=8
center_species_mask = np.where(info[:, 2] == sp_i)[0]
neighbor_species_mask = np.where(info[:, 2] == sp_j)[0]

In [364]:
pair_loc=[]
for i, (struct_i, atom_i) in enumerate(info[center_species_mask[:]][...,:-1]):
    for j, (struct_j, atom_j) in enumerate(info[neighbor_species_mask[:]][...,:-1]):
        if struct_i==struct_j:
            if [struct_i, int(atom_j), int(atom_i)] not in pair_loc:
                pair_loc.append([struct_i, int(atom_i), int(atom_j)])#+list(j[1:]))

In [42]:

for a1 in actual_global_species:
    for a2 in actual_global_species:
        

[1, 8]

In [33]:
rho0ij=compute_rho0ij(rascal_hypers, frames)

In [34]:
rho0ij.sparse

Labels([(1, 0, 0), (1, 1, 0), (1, 2, 0)],
       dtype=[('sigma', '<i4'), ('L', '<i4'), ('nu', '<i4')])

In [40]:
rhoi.sparse

Labels([(0, 1, 1), (0, 1, 8), (0, 8, 1), (0, 8, 8), (1, 1, 1), (1, 1, 8),
        (1, 8, 1), (1, 8, 8), (2, 1, 1), (2, 1, 8), (2, 8, 1), (2, 8, 8)],
       dtype=[('spherical_harmonics_l', '<i4'), ('center_species', '<i4'), ('neighbor_species', '<i4')])

In [39]:
rhoi.block(spherical_harmonics_l=1, center_species=1, neighbor_species=8).components

Labels([(-1,), ( 0,), ( 1,)], dtype=[('spherical_harmonics_m', '<i4')])

In [ ]:
acdc_nu1 = Descriptor(sparse = Labels(names=["sigma", "lam", "nu"], 
                                      values=np.asarray([[ 1, l, 1] for l in range(rascal_hypers["max_angular"]+1)], dtype=np.int32)), 
                      blocks = blocks
                     )

In [16]:
rho1ij=

Labels([(0,), (1,), (2,)], dtype=[('spherical_harmonics_l', '<i4')])

In [17]:
stop here 

SyntaxError: invalid syntax (2391973759.py, line 1)

In [ ]:
hypers=rascal_hypers

In [ ]:
#hypers = copy.deepcopy(self._hypers)
if hypers["compute_gradients"]:
    raise Exception("Pair expansion with gradient is not implemented")

# max_atoms = max([len(f) for f in frames])
# global_species = list(range(max_atoms))

# hypers["global_species"] = global_species
# hypers["expansion_by_species_method"] = "user defined"

# ijframes = []
# for f in frames:
#     ijf = f.copy()
#     ijf.numbers = global_species[:len(f)]
#     ijframes.append(ijf)

calculator = SphericalExpansion(**hypers)

# Step 2: move data around to follow the storage convention
sparse = Labels(
    names=["spherical_harmonics_l"],
    values=np.array(
        [
            [l]
            for l in range(hypers["max_angular"] + 1)                    
        ],
        dtype=np.int32,
    ),
)

features = Labels(
    names=["n"],
    values=np.array([[n] for n in range(hypers["max_radial"])], dtype=np.int32),
)

lm_slices = []
start = 0
for l in range(hypers["max_angular"] + 1):
    stop = start + 2 * l + 1
    lm_slices.append(slice(start, stop))
    start = stop

data = []
samples = []
for i, ijf in enumerate(ijframes):
    idata = calculator.transform(ijf).get_features(calculator).reshape(len(ijf), max_atoms, hypers["max_radial"], -1)
    nonzero = np.where( (idata**2).sum(axis=(2,3)) > 1e-20)
    data.append(idata[nonzero[0], nonzero[1]].reshape(len(nonzero[0]),hypers["max_radial"],-1) )
    samples.append( np.asarray( [nonzero[0]*0+i, nonzero[0], nonzero[1]] ).T )

data = np.concatenate(data)
samples = Labels(
        names=["structure", "center_i", "center_j"],
        values=np.concatenate(samples).astype(np.int32)
)
blocks = []
for (l,) in sparse:
    block_data = data[..., lm_slices[l]]
    block_data = block_data.swapaxes(1, 2)

    components = Labels(
        names=["spherical_harmonics_m"],
        values=np.array([[m] for m in range(-l, l + 1)], dtype=np.int32),
    )

    block = Block(
        values=np.copy(block_data),
        samples=samples,
        components=components,
        features=features,
    )

    blocks.append(block)
